# Introduction à la planification classique

## A. Présentation des technologies utilisées

### 1. Fast Downward

[Fast Downward](https://github.com/aibasel/downward) (cloner le dépôt) est un système de planification automatisée open-source développé en C++. Il implémente différents algorithmes de planification basés sur la recherche heuristique, tels que LAMA, Dijkstra, A*, etc. Fast Downward est utilisé pour résoudre des problèmes de planification dans divers domaines, tels que la planification de tâches, la planification logistique, la planification de ressources, etc.

### 2. Google OR-Tools

Google OR-Tools est une suite open-source de logiciels d'optimisation qui comprend des solvers pour les problèmes de programmation par contraintes (CSP), la satisfaction de contraintes booléennes (SAT), l'optimisation linéaire (LP), l'optimisation en nombres entiers (MIP), l'optimisation de réseau, et la planification de routes.
Dans le contexte de ce TP, nous nous intéresserons principalement aux fonctionnalités de planification de OR-Tools, qui fournissent une suite d'algorithmes et de méthodes pour résoudre des problèmes de planification tels que le problème du voyageur de commerce, le problème de sac à dos, et la planification de tâches.
OR-Tools peut être utilisé en C# via le package Nuget officiel. Il fournit une API C# qui permet d'utiliser les solvers et les méthodes d'optimisation de OR-Tools dans un projet .NET.


## B. Problèmes et domaines de planification

### 1. Qu’est-ce que la planification ?

La planification est une composante essentielle de l'intelligence artificielle (IA). Dans ce cadre, on peut définir la planification comme le processus permettant de déterminer une séquence d'actions qui mènera un agent d'un état initial à un état d'objectif. Ce concept est crucial dans diverses applications de l'IA, notamment dans la robotique, où il est souvent nécessaire d'établir un plan d'action pour un robot qui doit accomplir une série de tâches dans un ordre précis.

La planification en IA se base sur une représentation formalisée d'un problème de planification classique, qui est généralement exprimé comme un triplet : <I, G, D>. Ici, I représente l'état initial de l'agent, G est l'état d'objectif que l'agent veut atteindre, et D décrit la théorie du domaine, c'est-à-dire l'ensemble des actions que l'agent peut entreprendre pour modifier son environnement.

### 2. Représentation élémentaire : le modèle STRIPS

L'un des modèles de planification les plus fondamentaux en IA est STRIPS (Stanford Research Institute Problem Solver). Dans STRIPS, les actions sont formalisées avec une liste de préconditions et une liste d'effets. Les préconditions doivent être satisfaites pour que l'action puisse être effectuée, et les effets sont les conséquences de l'action sur l'état du monde.
Une action est définie par :
- Son nom
- Ses paramètres
- Ses préconditions
- Ses résultats

#### Exemple de STRIPS

Considérons un exemple simple dans le monde des blocs. Supposons que nous ayons un bras robotique qui peut effectuer un certain nombre d'actions spécifiques, comme prendre un bloc, le déposer, le placer sur un autre bloc, ou le retirer d'un autre bloc. Dans STRIPS, chaque action serait représentée avec une liste de préconditions et une liste d'effets. Par exemple, pour l'action de prendre un bloc, les préconditions pourraient inclure le fait que le bras soit libre et que le bloc soit accessible. Les effets de l'action seraient que le bras n'est plus libre (il tient le bloc) et que le bloc n'est plus accessible (il est dans la main du robot).

##### Quatre actions définies pour le monde des blocs

```strips
pickup(X)
P: gripping() ∧ clear(X) ∧ ontable(X)
A: gripping(X)
D: ontable(X) ∧ gripping()
putdown(X)
P: gripping(X)
A: ontable(X) ∧ gripping() ∧ clear(X)
D: gripping(X)
stack(X,Y)
P: gripping(X) ∧ clear(Y)
A: on(X,Y) ∧ gripping() ∧ clear(X)
D:gripping(X) ∧ clear(Y)
unstack(X,Y)
P: gripping() ∧ clear(X) ∧ on(X,Y)
A: gripping(X) ∧ clear(Y)
D: on(X,Y) ∧ gripping()
```

### 3. Les limites de STRIPS


STRIPS repose sur plusieurs hypothèses importantes qui simplifient le problème de planification, mais qui peuvent également limiter son applicabilité. Ces hypothèses comprennent l'idée que le monde est statique (à part les actions de l'agent), que toutes les actions sont déterministes et que l'agent a une connaissance parfaite du monde. Cependant, malgré sa utilité, STRIPS présente certaines limites.

#### Le problème du cadre

Le problème du cadre est l'un des problèmes qui peut se poser lors de l'utilisation de STRIPS. Il consiste à énumérer tout ce qui ne change pas dans le problème de planification. Cependant, de nos jours, ce problème est moins significatif car ce qui n'est pas explicitement listé comme changeant (via les listes d'ajout et de suppression d'effets) est supposé ne pas changer.

#### Le problème de qualification

Le problème de qualification est un autre défi dans la représentation des actions dans STRIPS. Il concerne la possibilité d'énumérer chaque précondition nécessaire pour qu'une action puisse être exécutée. Par exemple, dans le cas de l'action "prendre un bloc", il peut être nécessaire de spécifier que le bloc ne doit pas être collé à la table ou cloué à la table, etc. Ce problème reste une difficulté, mais une solution partielle consiste à hiérarchiser les préconditions en déterminant celles qui sont les plus importantes à atteindre.

#### Extensions de STRIPS : PDDL

Pour surmonter ces limites, le langage de définition du domaine de planification (PDDL) a été introduit. PDDL est une extension de STRIPS qui permet une plus grande flexibilité dans la représentation des problèmes de planification. Par exemple, PDDL permet de représenter les actions concurrentes, les actions non déterministes et les négations des effets des actions. De plus, PDDL offre des fonctionnalités supplémentaires, comme la possibilité de représenter les préférences et les contraintes temporelles.

### 4. PDDL et le problème du Gripper


Dans PDDL, chaque action ou opérateur est défini par son nom, ses paramètres, ses préconditions et ses effets. C'est une représentation spéciale conçue pour la planification qui simplifie considérablement le processus en faisant de nombreuses suppositions.


#### Domaine (domain.pddl)

```pddl
(define (domain gripper-strips)
  (:predicates (room ?r)
               (ball ?b)
               (gripper ?g)
               (at-robby ?r)
               (at ?b ?r)
               (free ?g)
               (carry ?o ?g))
  (:action move
           :parameters (?from ?to)
           :precondition (and (room ?from) (room ?to) (at-robby ?from))
           :effect (and (at-robby ?to) (not (at-robby ?from))))
  (:action pick
           :parameters (?obj ?room ?gripper)
           :precondition (and (ball ?obj) (room ?room) (gripper ?gripper) (at ?obj ?room) (at-robby ?room) (free ?gripper))
           :effect (and (carry ?obj ?gripper) (not (at ?obj ?room)) (not (free ?gripper))))
  (:action drop
           :parameters (?obj ?room ?gripper)
          

 :precondition (and (ball ?obj) (room ?room) (gripper ?gripper) (at-robby ?room) (carry ?obj ?gripper))
           :effect (and (at ?obj ?room) (free ?gripper) (not (carry ?obj ?gripper)))))
```

#### Problème (problem.pddl)

```pddl
(define (problem strips-gripper2)
  (:domain gripper-strips)
  (:objects rooma roomb ball1 ball2 left right)
  (:init (room rooma) (room roomb) (ball ball1) (ball ball2) (gripper left) (gripper right) (at-robby rooma) (free left) (free right) (at ball1 rooma) (at ball2 rooma))
  (:goal (at ball1 roomb)))
```


## C. Exécution de commandes avec Fast Downward

### Préparation de l'environnement
Nous allons créer une méthode en C# pour exécuter des commandes externes comme `fast-downward.py`. Cette méthode utilisera `System.Diagnostics.Process`.

In [1]:
using System;
using System.Diagnostics;
using System.IO;
using System.IO.Compression;
using System.Net.Http;
using System.Threading.Tasks;

public class FastDownwardRunner
{
    // Configuration: chemin vers Fast Downward (auto-detecte ou telecharge si necessaire)
    private static string _fastDownwardPath = null;

    // Repertoire par defaut pour Fast Downward (relatif au notebook)
    private static readonly string DEFAULT_FD_DIR = Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.UserProfile),
        ".local", "fast-downward"
    );

    // URL de telechargement (release GitHub)
    private const string FD_DOWNLOAD_URL = "https://github.com/aibasel/downward/archive/refs/tags/release-24.06.1.zip";
    private const string FD_VERSION = "24.06.1";

    public static string FastDownwardPath
    {
        get
        {
            if (_fastDownwardPath == null)
            {
                _fastDownwardPath = FindOrDownloadFastDownward();
            }
            return _fastDownwardPath;
        }
        set { _fastDownwardPath = value; }
    }

    private string BasePath { get; }

    public FastDownwardRunner(string basePath = null)
    {
        // Si basePath n'est pas specifie, utiliser le repertoire de Fast Downward
        BasePath = basePath ?? Path.GetDirectoryName(FastDownwardPath);
    }

    private static string FindOrDownloadFastDownward()
    {
        // 1. Verifier la variable d'environnement
        var envPath = Environment.GetEnvironmentVariable("FAST_DOWNWARD_PATH");
        if (!string.IsNullOrEmpty(envPath) && File.Exists(envPath))
        {
            Console.WriteLine($"Fast Downward trouve via FAST_DOWNWARD_PATH: {envPath}");
            return envPath;
        }

        // 2. Verifier le repertoire par defaut
        var defaultPath = Path.Combine(DEFAULT_FD_DIR, $"downward-release-{FD_VERSION}", "fast-downward.py");
        if (File.Exists(defaultPath))
        {
            // Verifier si la compilation a ete effectuee
            var buildsDir = Path.Combine(Path.GetDirectoryName(defaultPath), "builds");
            if (Directory.Exists(buildsDir) && Directory.GetDirectories(buildsDir).Length > 0)
            {
                Console.WriteLine($"Fast Downward trouve et compile: {defaultPath}");
                return defaultPath;
            }
            else
            {
                Console.WriteLine($"Fast Downward trouve mais non compile. Lancement de la compilation...");
                CompileFastDownward(Path.GetDirectoryName(defaultPath));
                return defaultPath;
            }
        }

        // 3. Telecharger automatiquement
        Console.WriteLine($"Fast Downward non trouve. Telechargement en cours...");
        return DownloadFastDownward().Result;
    }

    private static async Task<string> DownloadFastDownward()
    {
        Directory.CreateDirectory(DEFAULT_FD_DIR);
        var zipPath = Path.Combine(DEFAULT_FD_DIR, $"fast-downward-{FD_VERSION}.zip");
        var extractPath = DEFAULT_FD_DIR;

        using (var client = new HttpClient())
        {
            client.Timeout = TimeSpan.FromMinutes(10);
            Console.WriteLine($"Telechargement depuis {FD_DOWNLOAD_URL}...");

            var response = await client.GetAsync(FD_DOWNLOAD_URL);
            response.EnsureSuccessStatusCode();

            using (var fs = new FileStream(zipPath, FileMode.Create))
            {
                await response.Content.CopyToAsync(fs);
            }
        }

        Console.WriteLine("Extraction de l'archive...");
        ZipFile.ExtractToDirectory(zipPath, extractPath, true);
        File.Delete(zipPath);

        var fdDir = Path.Combine(extractPath, $"downward-release-{FD_VERSION}");
        var fdPath = Path.Combine(fdDir, "fast-downward.py");

        if (File.Exists(fdPath))
        {
            Console.WriteLine($"Fast Downward extrait: {fdPath}");

            // Compilation automatique
            Console.WriteLine("Lancement de la compilation automatique...");
            CompileFastDownward(fdDir);

            return fdPath;
        }

        throw new Exception("Echec de l'installation de Fast Downward");
    }

    private static void CompileFastDownward(string fdDir)
    {
        Console.WriteLine($"Compilation de Fast Downward dans {fdDir}...");
        Console.WriteLine("Cette operation peut prendre plusieurs minutes...");

        var buildScript = Path.Combine(fdDir, "build.py");
        if (!File.Exists(buildScript))
        {
            Console.WriteLine($"ERREUR: Script de compilation non trouve: {buildScript}");
            return;
        }

        var processStartInfo = new ProcessStartInfo
        {
            FileName = "python",
            Arguments = $"\"{buildScript}\"",
            RedirectStandardOutput = true,
            RedirectStandardError = true,
            UseShellExecute = false,
            CreateNoWindow = true,
            WorkingDirectory = fdDir
        };

        try
        {
            using (var process = Process.Start(processStartInfo))
            {
                if (process != null)
                {
                    // Lire la sortie en temps reel pour montrer la progression
                    process.OutputDataReceived += (sender, e) => {
                        if (!string.IsNullOrEmpty(e.Data))
                            Console.WriteLine(e.Data);
                    };
                    process.ErrorDataReceived += (sender, e) => {
                        if (!string.IsNullOrEmpty(e.Data))
                            Console.WriteLine($"[stderr] {e.Data}");
                    };

                    process.BeginOutputReadLine();
                    process.BeginErrorReadLine();

                    // Attendre la fin de la compilation (timeout 30 minutes)
                    var completed = process.WaitForExit(30 * 60 * 1000);

                    if (!completed)
                    {
                        process.Kill();
                        Console.WriteLine("ERREUR: Timeout de compilation (30 minutes)");
                        return;
                    }

                    if (process.ExitCode == 0)
                    {
                        Console.WriteLine("Compilation de Fast Downward terminee avec succes!");
                    }
                    else
                    {
                        Console.WriteLine($"ATTENTION: La compilation s'est terminee avec le code {process.ExitCode}");
                        Console.WriteLine("Fast Downward pourrait ne pas fonctionner correctement.");
                    }
                }
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"ERREUR lors de la compilation: {ex.Message}");
            Console.WriteLine("Vous pouvez compiler manuellement avec:");
            Console.WriteLine($"  cd \"{fdDir}\" && python build.py");
        }
    }

    public void RunFastDownward(string domainFileRelative, string problemFileRelative, string additionalArgs = "")
    {
        var domainFile = Path.Combine(BasePath, domainFileRelative);
        var problemFile = string.IsNullOrEmpty(problemFileRelative) ? "" : Path.Combine(BasePath, problemFileRelative);

        // Afficher le debut du fichier de domaine
        if (!string.IsNullOrEmpty(domainFileRelative))
            DisplayFileStart(domainFile, "Domaine");

        // Afficher le debut du fichier de probleme
        if (!string.IsNullOrEmpty(problemFileRelative))
            DisplayFileStart(problemFile, "Probleme");

        var args = string.IsNullOrEmpty(problemFile)
            ? $"\"{FastDownwardPath}\" \"{domainFile}\" {additionalArgs}"
            : $"\"{FastDownwardPath}\" \"{domainFile}\" \"{problemFile}\" {additionalArgs}";

        var processStartInfo = new ProcessStartInfo
        {
            FileName = "python",
            Arguments = args,
            RedirectStandardOutput = true,
            RedirectStandardError = true,
            UseShellExecute = false,
            CreateNoWindow = true,
            WorkingDirectory = BasePath
        };

        using (var process = Process.Start(processStartInfo))
        {
            if (process != null)
            {
                using (var reader = process.StandardOutput)
                {
                    string result = reader.ReadToEnd();
                    Console.WriteLine(result);
                }
                var errors = process.StandardError.ReadToEnd();
                if (!string.IsNullOrEmpty(errors))
                    Console.WriteLine($"Erreurs: {errors}");
            }
        }

        // Afficher le contenu du fichier sas_plan
        var sasPlanFile = Path.Combine(BasePath, "sas_plan");
        if (File.Exists(sasPlanFile))
        {
            Console.WriteLine("Contenu de sas_plan:");
            Console.WriteLine(File.ReadAllText(sasPlanFile));
        }
    }

    private void DisplayFileStart(string filePath, string fileType)
    {
        if (File.Exists(filePath))
        {
            Console.WriteLine($"Debut du fichier {fileType} ({filePath}):");
            using (var reader = new StreamReader(filePath))
            {
                for (int i = 0; i < 50; i++)
                {
                    if (reader.EndOfStream) break;
                    Console.WriteLine(reader.ReadLine());
                }
            }
        }
        else
        {
            Console.WriteLine($"Le fichier {fileType} ({filePath}) n'a pas ete trouve.");
        }
    }
}

// Initialisation - le chemin sera auto-detecte ou Fast Downward sera telecharge et compile
Console.WriteLine("Initialisation de Fast Downward...");
Console.WriteLine($"Chemin: {FastDownwardRunner.FastDownwardPath}");
var runner = new FastDownwardRunner();
Console.WriteLine("Runner initialise.");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Initialisation de Fast Downward...


Fast Downward trouve et compile: C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\fast-downward.py


Chemin: C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\fast-downward.py


Runner initialise.


### Interprétation de l'initialisation

**Objectif** : Cette cellule met en place la classe `FastDownwardRunner` qui automatise l'installation, la compilation et l'exécution de Fast Downward.

**Points clés** :
- **Auto-détection** : Le système cherche Fast Downward dans la variable d'environnement `FAST_DOWNWARD_PATH`, puis dans `~/.local/fast-downward`
- **Téléchargement automatique** : Si Fast Downward n'est pas trouvé, il est automatiquement téléchargé depuis GitHub (release 24.06.1)
- **Compilation automatique** : Le script `build.py` est exécuté pour compiler les binaires C++ de Fast Downward
- **Affichage des fichiers** : Les méthodes permettent d'afficher les fichiers PDDL (domaine et problème) avant l'exécution

> **Note technique** : La compilation peut prendre plusieurs minutes lors de la première exécution. Un timeout de 30 minutes est configuré pour gérer les compilations longues.

## III. Syntaxe de ligne de commande de Fast Downward

Fast Downward propose une interface en ligne de commande flexible pour paramétrer ses différents composants. Les paramètres peuvent être spécifiés par mot-clé ou par position, et certains ont des valeurs par défaut. Voici quelques exemples de commandes et options disponibles.

Fast Downward propose une fonctionnalité permettant de spécifier différentes options en fonction des propriétés du fichier d'entrée, comme la sélection des heuristiques à utiliser en fonction du coût unitaire ou non unitaire des actions. Cette option est conditionnée par des sélecteurs tels que --if-unit-cost.

Fast Downward offre également la possibilité de définir des variables pour des objets qui doivent être utilisés à plusieurs reprises, par exemple une heuristique ou une LandmarkFactory. Ces variables peuvent être définies avec let(name, definition, expression).

Les options de commande comprennent des options pour sélectionner les composants du planificateur à exécuter (translation, recherche), des limites de temps et de mémoire, des options pour le fichier de plan et de nombreuses autres options.


### Options de traduction

Fast Downward permet une grande flexibilité dans la façon dont le problème de planification est traduit en une tâche de planification abstraite qui peut être résolue par l'algorithme de recherche. Les options de traduction permettent de spécifier des options telles que l'encodage complet et d'autres options spécifiques à la tâche de planification.

### Interprétation de la traduction avec encodage complet

Cette commande exécute Fast Downward sur le problème du **Gripper** avec l'option `--full-encoding`, qui génère une représentation complète de la tâche de planification.

**Résultat attendu** :
- Traduction du PDDL vers le format SAS+ interne de Fast Downward
- Génération des fichiers intermédiaires (`output.sas`)
- Affichage des statistiques de traduction (nombre de variables, domaines, opérateurs)

> **Astuce** : L'encodage complet peut augmenter la taille de la représentation mais facilite certains types d'heuristiques.

In [2]:
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", 
    "misc/tests/benchmarks/gripper/prob01.pddl", "--translate-options --full-encoding");

Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



### Interprétation de la recherche avec l'heuristique FF

Cette cellule démontre l'utilisation de **l'heuristique FF (Fast Forward)** combinée avec une recherche **eager greedy**.

**Composants de la commande** :
- `--heuristic "h=ff()"` : Définit une heuristique FF nommée `h`
- `--search "eager_greedy([h])"` : Utilise une recherche gloutonne qui évalue immédiatement tous les successeurs

**Stratégie** :
| Algorithme | Heuristique | Garantie d'optimalité |
|------------|-------------|----------------------|
| Eager Greedy | FF | Non (mais rapide) |

L'heuristique FF estime le coût en relaxant les effets négatifs (relaxation de suppression). Elle est **non admissible** mais très efficace en pratique.

### Options de recherche

Les options de recherche spécifient l'algorithme de recherche à utiliser pour résoudre la tâche de planification. Fast Downward supporte une grande variété d'algorithmes de recherche, y compris A*, chaînage avant, chaînage arrière et planification d'ordre partiel. 

Les options de recherche permettent également de spécifier des options spécifiques à l'algorithme de recherche, telles que les heuristiques à utiliser.


### Utilisation des limites de ressources

Cette cellule exécute Fast Downward avec des contraintes explicites de **temps** et de **mémoire** pour les phases de traduction et de recherche.

**Paramètres configurés** :

| Phase | Limite de temps | Limite mémoire |
|-------|-----------------|----------------|
| Traduction | 300s (5 min) | 4000 MB |
| Recherche | 500s (8.3 min) | 8000 MB |

> **Cas d'usage** : Ces limites sont essentielles pour les compétitions de planification ou pour éviter qu'un problème difficile ne monopolise les ressources indéfiniment.

In [3]:
// Exemple d'exécution avec les options de recherche
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", 
"misc/tests/benchmarks/gripper/prob01.pddl", 
"--search-options --heuristic \"h=ff()\" --search \"eager_greedy([h])\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



### Pourquoi exécuter deux fois la même commande ?

Cette cellule est identique à la précédente et semble redondante. Dans un contexte pédagogique, on pourrait :
- **Comparer différentes limites** pour observer l'impact sur les performances
- **Tester la reproductibilité** des résultats

**Suggestion** : Modifier les limites pour illustrer l'impact sur le temps de résolution ou le nombre de nœuds explorés.

### Utilisation d'un alias

Fast Downward permet l'utilisation d'alias pour spécifier des configurations de planification prédéfinies. Ces alias peuvent être spécifiés à l'aide de l'option --alias sur la ligne de commande.


In [4]:
// Exemple d'exécution avec des limites de temps et de mémoire
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "", "--translate-time-limit 300 --translate-memory-limit 4000 --search-time-limit 500 --search-memory-limit 8000");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Erreurs: Traceback (most recent call last):
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\fast-downward.py", line 5, in <module>
    main()
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\main.py", line 14, in main
    args = arguments.parse_args()
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 501, in parse_args
    _set_components_and_inputs(parser, args)
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 239, in _set_components_and_inputs
    _set_components_automatically(parser, args)
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 215, in _set_components_automatically
    if len(args.filenames) == 1 and _looks_like_search_input(args.filenames[0]):
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jsboi\.local\fa

### Interprétation de l'exécution avec portfolio

Cette commande illustre l'utilisation d'un **portfolio de configurations**, une stratégie avancée utilisée dans les compétitions de planification.

**Principe** :
- Le fichier `seq_opt_fdss_1.py` contient plusieurs configurations de planification
- Chaque configuration est exécutée séquentiellement jusqu'à trouver une solution
- Le temps total est limité à 30 minutes

**Avantages** :
- **Robustesse** : Différentes configurations excellent sur différents types de problèmes
- **Optimisation** : Combine les forces de multiples heuristiques

> **Note** : Les portfolios sont la stratégie standard des planificateurs performants aux compétitions IPC (International Planning Competition).

### Limites de temps et de mémoire

Fast Downward permet de spécifier des limites de temps et de mémoire pour les différentes composantes du planificateur. Ces limites peuvent être spécifiées à l'aide des options de ligne de commande --translate-time-limit, --translate-memory-limit, --search-time-limit, et --search-memory-limit.


### Résultat avec l'heuristique additive

**Caractéristiques** :
- **Non admissible** : Peut surestimer le coût (la solution peut ne pas être optimale)
- **Rapide à calculer** : Somme simple des coûts des sous-buts
- **Opérateurs préférés** : Identifie les actions utiles

L'heuristique additive tend à **surestimer** car elle suppose que tous les sous-buts sont indépendants, ce qui n'est souvent pas le cas (ils peuvent interagir positivement).

### Exploration des heuristiques de Fast Downward

Dans les cellules suivantes, nous allons explorer systématiquement différentes **heuristiques** implémentées dans Fast Downward. Chaque heuristique possède des propriétés différentes :

| Propriété | Signification |
|-----------|---------------|
| **Admissible** | Ne surestime jamais le coût réel (garantit optimalité avec A*) |
| **Cohérente** | Respecte l'inégalité triangulaire (admissibilité + propriété plus forte) |
| **Sûre** | Fonctionne correctement avec axiomes et effets conditionnels |
| **Opérateurs préférés** | Identifie des actions prometteuses pour accélérer la recherche |

Nous commençons avec l'**heuristique additive (add)**, qui combine les coûts estimés des sous-buts de manière additive.

### Résultat avec l'heuristique aveugle

**Caractéristiques** :
- **Admissible et cohérente** : Garantit l'optimalité avec A*
- **Très simple** : Retourne le coût minimal d'action (souvent 1)
- **Pas d'opérateurs préférés** : Aucune guidance

Cette heuristique est équivalente à une **recherche de Dijkstra** (ou recherche en largeur si tous les coûts sont unitaires). Elle explore exhaustivement sans guidance, ce qui est très lent mais garantit l'optimalité.

In [5]:
// Exemple d'exécution avec des limites de temps et de mémoire
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "", "--translate-time-limit 300 --translate-memory-limit 4000 --search-time-limit 500 --search-memory-limit 8000");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Erreurs: Traceback (most recent call last):
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\fast-downward.py", line 5, in <module>
    main()
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\main.py", line 14, in main
    args = arguments.parse_args()
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 501, in parse_args
    _set_components_and_inputs(parser, args)
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 239, in _set_components_and_inputs
    _set_components_automatically(parser, args)
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 215, in _set_components_automatically
    if len(args.filenames) == 1 and _looks_like_search_input(args.filenames[0]):
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jsboi\.local\fa

### Résultat avec l'heuristique context-enhanced additive

**Amélioration de l'heuristique additive** :
- Prend en compte les **interactions entre sous-buts**
- Plus précise que l'heuristique additive simple
- Toujours **non admissible**, mais meilleure guidance en pratique

L'heuristique CEA analyse le contexte dans lequel les actions sont appliquées pour affiner l'estimation des coûts.

### Validation de plan

Fast Downward peut également valider les plans générés en utilisant l'outil de validation VAL.

### Résultat avec l'heuristique merge-and-shrink

**Technique sophistiquée** basée sur l'abstraction :
- **Merge** : Fusion de variables de planification
- **Shrink** : Réduction de l'espace d'états par bisimulation

**Propriétés** :
- **Admissible et cohérente** : Optimalité garantie
- **Configurable** : Nombreuses stratégies de fusion et réduction
- **Coûteuse** : Précalcul parfois long mais estimation très précise

Cette heuristique est particulièrement efficace sur les problèmes avec de nombreuses interactions entre variables.

### Utilisation d'un portfolio

Fast Downward supporte l'utilisation de portfolios de configurations de planification. Un portfolio est un ensemble de configurations de planification qui sont exécutées en séquence ou en parallèle pour résoudre une tâche de planification. L'utilisation d'un portfolio peut être spécifiée à l'aide de l'option --portfolio sur la ligne de commande.


### Transition vers les heuristiques classiques

Les cellules suivantes explorent des heuristiques plus classiques et largement utilisées dans la littérature de planification automatique. Nous commençons par l'**heuristique de graphe causal (cg)**, qui analyse les dépendances causales entre variables.

In [6]:
// Exemple d'exécution avec un portfolio
runner.RunFastDownward("", "", "--portfolio driver/portfolios/seq_opt_fdss_1.py --search-time-limit 30m output.sas");


Erreurs: Traceback (most recent call last):
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\fast-downward.py", line 5, in <module>
    main()
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\main.py", line 14, in main
    args = arguments.parse_args()
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 501, in parse_args
    _set_components_and_inputs(parser, args)
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 239, in _set_components_and_inputs
    _set_components_automatically(parser, args)
  File "C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\driver\arguments.py", line 215, in _set_components_automatically
    if len(args.filenames) == 1 and _looks_like_search_input(args.filenames[0]):
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jsboi\.local\fa

### Résultat avec l'heuristique de graphe causal

**Analyse des dépendances causales** :
- Construit un **graphe de causalité** entre variables
- Estime le coût en analysant les chemins dans ce graphe
- **Opérateurs préférés** : Identifie les actions qui résolvent les sous-buts critiques

**Limitations** :
- **Non admissible** : Peut surestimer ou sous-estimer
- **Axiomes** : Support rudimentaire

Cette heuristique est efficace sur les problèmes où les dépendances causales sont structurées.


### Heuristiques avancées de Fast Downward

L'efficacité de Fast Downward réside dans son usage d'heuristiques de pointe pour guider la recherche de solutions aux problèmes de planification. Quelques-unes de ces heuristiques avancées sont décrites ci-dessous.

Chacune de ces heuristiques possède des avantages et des limites en termes d'efficacité, d'admissibilité, de cohérence, et d'identification d'opérateurs privilégiés. Le choix de l'heuristique à utiliser dépend donc du problème de planification spécifique à résoudre.


#### Additive Heuristic

L'heuristique additive (add) est une méthode permettant de combiner les estimations de plusieurs heuristiques pour une meilleure estimation. Elle n'est ni admissible, ni cohérente, mais identifie les opérateurs privilégiés. 

Les options de cette heuristique incluent le niveau de verbosité, la transformation de la tâche, et le cache des estimations.

### Résultat avec l'heuristique FF (Fast Forward)

**Une des heuristiques les plus populaires** :
- Basée sur la **relaxation de suppression** (ignore les effets négatifs)
- Construit un graphe de planification relaxé pour estimer le coût
- **Opérateurs préférés** : Actions apparaissant dans le plan relaxé

**Performance** :
- **Rapide à calculer** : Complexité polynomiale
- **Non admissible** mais très efficace en pratique
- **Référence** dans les compétitions de planification

L'heuristique FF a révolutionné la planification automatique dans les années 2000 en permettant de résoudre des problèmes auparavant intractables.

In [7]:
// Exemple d'exécution avec l'heuristique additive
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(add ())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



### Résultat avec l'heuristique goalcount

**Heuristique la plus simple** :
- Compte simplement le **nombre de sous-buts non atteints**
- Ignore complètement les coûts des actions
- **Très rapide** mais peu informative

**Utilité** :
- Référence de base pour comparer d'autres heuristiques
- Utile pour les problèmes où tous les sous-buts sont également difficiles

Cette heuristique donne essentiellement une borne inférieure triviale sur le nombre d'actions restantes.

#### Blind Heuristic

L'heuristique aveugle (blind) retourne le coût de l'action la moins chère pour les états non-cibles et 0 pour les états cibles. Elle supporte les coûts d'action, les effets conditionnels, et les axiomes. C'est une heuristique admissible, cohérente et sûre, mais elle ne permet pas d'identifier des opérateurs privilégiés.

### Résultat avec l'heuristique h^m

**Heuristique basée sur les ensembles de faits** :
- Considère les interactions entre **m faits simultanément**
- Plus m est grand, plus l'heuristique est précise (mais coûteuse)
- **Admissible et cohérente** pour m suffisamment grand

**Compromis** :
| m | Précision | Complexité |
|---|-----------|------------|
| 1 | Faible | O(n) |
| 2 | Moyenne | O(n²) |
| 3+ | Élevée | Exponentielle |

En pratique, m=1 ou m=2 sont les plus utilisés. Pour m → ∞, on obtient l'heuristique h* (optimale).

In [8]:
// Exemple d'exécution avec l'heuristique aveugle
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(blind())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



### Résultat avec l'heuristique hmax

**Variante conservative de l'heuristique additive** :
- Prend le **maximum** des coûts des sous-buts au lieu de la somme
- **Admissible** : Ne surestime jamais le coût
- Plus précise que goalcount, moins que h^m

**Formule** :
```
h_add(s) = Σ coût(sous-but_i)  → peut surestimer
h_max(s) = max(coût(sous-but_i))  → admissible mais moins informative
```

L'heuristique hmax est utile quand on cherche l'optimalité avec une heuristique simple et rapide.


#### Context-enhanced Additive Heuristic

L'heuristique context-enhanced additive (cea) est une heuristique non admissible, non cohérente et non sûre, mais qui identifie les opérateurs privilégiés. Comme pour les heuristiques précédentes, elle offre des options pour le niveau de verbosité, la transformation de la tâche, et le cache des estimations.

### Transition vers les techniques avancées

Les sections suivantes explorent des techniques de planification plus sophistiquées qui combinent plusieurs approches :
- **Relaxations et repères** (landmarks)
- **Adaptation dynamique des coûts**
- **Bases de données de motifs** (Pattern Databases)

Ces techniques représentent l'état de l'art en planification classique et sont utilisées dans les planificateurs gagnants des compétitions.

In [9]:
// Exemple d'exécution avec l'heuristique context-enhanced additive
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(cea())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



### Résultat avec l'heuristique LM-cut

**Technique de pointe basée sur les landmarks** :
- Un **landmark** est un fait qui doit obligatoirement être vrai dans tout plan solution
- **LM-cut** identifie des ensembles de landmarks et calcule une borne inférieure optimale
- **Admissible** : Garantit l'optimalité

**Principe** :
1. Identifier un "cut" (coupure) dans le graphe de relaxation
2. Chaque cut correspond à un ensemble d'actions dont au moins une doit être exécutée
3. Sommer les coûts minimaux de chaque cut

Cette heuristique est considérée comme l'une des plus précises pour la planification optimale, au prix d'un temps de calcul plus élevé.

#### Merge-and-shrink Heuristic

L'heuristique merge-and-shrink (merge_and_shrink) est basée sur l'algorithme décrit dans plusieurs articles scientifiques. Cette heuristique est configurable avec plusieurs stratégies de fusion et de réduction. Elle supporte les coûts d'action et les effets conditionnels, mais pas les axiomes. Elle est admissible, cohérente, sûre, mais ne permet pas d'identifier des opérateurs privilégiés.

### Résultat avec l'adaptation de coûts

**Transformation de la tâche** : Cette technique modifie dynamiquement les coûts des actions pour améliorer l'heuristique.

**Options d'adaptation** :

| Mode | Description |
|------|-------------|
| `normal` | Coûts originaux (pas de transformation) |
| `one` | Tous les coûts deviennent 1 (optimal pour longueur de plan) |
| `plusone` | Tous les coûts augmentent de 1 |

**Intérêt** :
- `one` : Utile pour minimiser le nombre d'actions
- `plusone` : Évite les cycles à coût nul dans certains domaines

Cette cellule combine LM-cut avec la transformation `one`, optimisant pour le plan le plus court.

In [10]:
// Exemple d'exécution avec l'heuristique merge-and-shrink
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(merge_and_shrink ())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



### Résultat avec les Pattern Databases (iPDB)

**Concept des Pattern Databases** :
- Un **pattern** est un sous-ensemble de variables du problème
- Une **PDB** précalcule les coûts optimaux pour toutes les combinaisons de valeurs du pattern
- Utilisé comme table de lookup pendant la recherche

**Incrémental (iPDB)** :
- Génère des patterns dynamiquement pendant la recherche
- Ajoute de nouveaux patterns quand les patterns existants ne sont pas assez informatifs

**Avantages** :
- **Admissible** : Garantit l'optimalité
- **Précis** : Stocke les coûts exacts pour les sous-problèmes
- **Incrémental** : S'adapte au problème spécifique

Cette technique est particulièrement efficace sur les problèmes avec une structure factorisée.

#### Operator-counting Heuristic

L'heuristique operator-counting (operatorcounting) est une heuristique qui calcule un programme linéaire à chaque état. Elle est admissible, cohérente, sûre, mais ne permet pas d'identifier des opérateurs privilégiés. Elle supporte les coûts d'action, mais pas les effets conditionnels ni les axiomes, en raison de la limitation des générateurs de contraintes actuellement implémentés.

### Résultat avec merge-and-shrink avancé

Cette configuration illustre une utilisation sophistiquée de **merge-and-shrink** avec plusieurs paramètres :

**Paramètres utilisés** :

| Paramètre | Valeur | Description |
|-----------|--------|-------------|
| `merge_strategy` | `merge_precomputed` | Utilise un arbre de fusion précalculé |
| `merge_tree` | `linear(reverse_level)` | Fusion linéaire en ordre inverse de niveau |
| `shrink_strategy` | `shrink_bisimulation(greedy=true)` | Réduction par bisimulation gloutonne |
| `label_reduction` | `exact(before_shrinking=true)` | Réduction exacte des étiquettes avant réduction |

**Compromis** :
- **Précalcul** : Plus long au démarrage
- **Précision** : Heuristique très informative
- **Mémoire** : Peut consommer beaucoup de RAM sur gros problèmes

Cette configuration est typique des planificateurs optimaux dans les compétitions.

## Conclusion et synthèse

### Récapitulatif des heuristiques explorées

Ce notebook a présenté un panorama complet des heuristiques de Fast Downward :

| Heuristique | Admissible | Rapide | Opérateurs préférés | Meilleur usage |
|-------------|------------|--------|---------------------|----------------|
| **blind** | ✓ | ✓ | ✗ | Référence de base |
| **goalcount** | ✗ | ✓✓ | ✗ | Problèmes simples |
| **add** | ✗ | ✓ | ✓ | Recherche rapide non optimale |
| **cea** | ✗ | ✓ | ✓ | Amélioration de add |
| **cg** | ✗ | ✓ | ✓ | Problèmes avec dépendances causales |
| **ff** | ✗ | ✓ | ✓ | Recherche gloutonne efficace |
| **hmax** | ✓ | ✓ | ✗ | Recherche optimale simple |
| **h^m** | ✓ (m grand) | ✗ | ✗ | Recherche optimale avec interactions |
| **lmcut** | ✓ | ✗ | ✗ | Recherche optimale précise |
| **merge_and_shrink** | ✓ | ✗ | ✗ | Problèmes factorisés |
| **iPDB** | ✓ | ✗ | ✗ | Problèmes avec structure |

### Stratégies de sélection

**Pour la planification optimale** (garantir le plan le moins coûteux) :
- `lmcut` : Meilleur compromis précision/temps
- `merge_and_shrink` : Si le problème a une structure factorisée
- `iPDB` : Si suffisamment de mémoire disponible

**Pour la planification satisfiable** (trouver rapidement un plan) :
- `ff` + `eager_greedy` : Stratégie standard, très efficace
- `cea` + `lazy_greedy` : Alternative robuste
- `cg` : Pour problèmes avec dépendances causales fortes

**Pour les compétitions** :
- Utiliser un **portfolio** qui combine plusieurs configurations
- Tester différentes heuristiques sur des benchmarks représentatifs

### Techniques avancées couvertes

- **Relaxations** : Simplifications du problème (suppression d'effets négatifs)
- **Landmarks** : Identification de faits obligatoires
- **Pattern Databases** : Précalcul de coûts pour sous-problèmes
- **Adaptation de coûts** : Transformation dynamique pour améliorer l'heuristique
- **Merge-and-shrink** : Abstraction par fusion et réduction

### Ressources pour aller plus loin

- [Documentation officielle Fast Downward](https://www.fast-downward.org/)
- [Compétitions IPC](https://www.icaps-conference.org/competitions/)
- [Benchmarks PDDL](https://github.com/aibasel/downward-benchmarks)
- Articles fondateurs : FF (Hoffmann & Nebel, 2001), LM-cut (Helmert & Domshlak, 2009)

Fast Downward reste l'un des planificateurs les plus performants et flexibles pour la planification classique, grâce à sa riche collection d'heuristiques et de techniques d'optimisation.

#### Heuristique de Graphique Causal (cg)

L'heuristique de Graphique Causal (cg) est configurable avec plusieurs paramètres comme la taille maximale du cache (max_cache_size), la verbosité (verbosity), la transformation de tâche (transform) et la mise en cache des estimations (cache_estimates). Elle supporte les coûts d'action, les effets conditionnels et les axiomes, bien que le traitement des axiomes puisse être rudimentaire. Elle n'est ni admissible, ni cohérente, ni sûre, mais elle prend en charge les opérateurs préférés.

In [11]:
// Exemple d'exécution avec l'heuristique de Graphique Causal
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(cg())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



#### Heuristique FF

L'heuristique FF partage de nombreuses caractéristiques avec l'heuristique de Graphique Causal, y compris la prise en charge des coûts d'action, des effets conditionnels et des axiomes. Elle est sûre pour les tâches sans axiomes et supporte également les opérateurs préférés. Comme l'heuristique de Graphique Causal, elle n'est ni admissible, ni cohérente.

In [12]:
// Exemple d'exécution avec l'heuristique FF
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(ff())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



#### Heuristique de Compte d'Objectif (goalcount)

L'heuristique de Compte d'Objectif ignore les coûts d'action par conception, mais supporte les effets conditionnels et les axiomes. Elle est sûre, mais n'est ni admissible, ni cohérente, et ne supporte pas les opérateurs préférés.

In [13]:
// Exemple d'exécution avec l'heuristique de Compte d'Objectif
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(goalcount())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



#### Heuristique h^m

L'heuristique h^m, où 'm' est la taille du sous-ensemble, supporte les coûts d'action mais ignore les effets conditionnels et les axiomes. Elle est admissible, cohérente et sûre pour les tâches sans effets conditionnels ou axiomes, mais ne prend pas en charge les opérateurs préférés.

In [14]:
// Exemple d'exécution avec l'heuristique h^m
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(hm())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



#### Heuristique Max (hmax)

L'heuristique Max est similaire à l'heuristique h^m en ce sens qu'elle supporte les coûts d'action et les effets conditionnels, et peut gérer les axiomes à un degré limité. Elle est admissible, cohérente et sûre pour les tâches sans axiomes, mais ne prend pas en charge les opérateurs préférés.

In [15]:
// Exemple d'exécution avec l'heuristique Max
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(hmax())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



#### Heuristique de Partage des Coûts des Points de Repère (landmark_cost_partitioning)

L'heuristique de Partage des Coûts des Points de Repère est une heuristique admissible qui utilise un ensemble de points de repère pour établir des coûts. Elle supporte les coûts d'action et les effets conditionnels, mais pas les axiomes. Cette heuristique est sûre et peut utiliser des opérateurs préférés si cela est activé.

### Utilisation avancée de Fast Downward

Fast Downward est un outil de planification classique flexible et puissant qui offre de nombreuses fonctionnalités avancées pour optimiser et personnaliser le processus de planification. Pour tirer pleinement parti de ces fonctionnalités, il est important de comprendre comment elles fonctionnent et comment les utiliser efficacement. Voici une description détaillée de quelques-unes de ces fonctionnalités.

#### Relaxations et repères

Fast Downward propose des fonctionnalités pour travailler avec des relaxations de suppression et des repères. La relaxation de suppression peut être utile pour calculer l'heuristique optimale h+. Fast Downward calcule également un ensemble de repères pour chaque état à l'aide de la méthode LM-cut, qui peut être très utile pour guider la recherche de planification.

In [16]:
// Exemple d'exécution avec la relaxation LM-cut
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(lmcut())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



#### Adaptation des coûts des actions

Fast Downward offre un moyen de moduler les coûts des actions via la transformation de la tâche racine, nommée "Cost-adapted task". Cette fonctionnalité offre trois options d'adaptation des coûts : "normal", "one" et "plusone". La capacité à adapter les coûts des actions peut avoir un impact significatif sur la qualité des plans produits.

In [17]:
// Exemple d'exécution avec l'adaptation des coûts des actions
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(lmcut(transform=adapt_costs(one)))\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.




#### Optimisation post-hoc et équations d'état

Fast Downward peut utiliser des bases de données de motifs (PDB) pour calculer des contraintes d'optimisation post-hoc. Ces contraintes peuvent aider à maximiser l'utilité des PDB pour la planification classique. De plus, Fast Downward ajoute une contrainte permanente pour chaque fait qui tient compte du changement net du fait.

In [18]:
// Exemple d'exécution avec l'optimisation post-hoc et les équations d'état
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(ipdb())\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



#### Réduction des étiquettes

Fast Downward offre une technique de réduction d'étiquettes, nommée "Exact Generalized Label Reduction". Cette technique peut être appliquée avant le rétrécissement et avant la fusion des systèmes de transitions.

#### Fabriques de repères et fonctions de scoring

Fast Downward propose divers types de fabriques de repères et plusieurs fonctions de scoring. Les repères peuvent être utilisés pour guider la recherche de planification, tandis que les fonctions de scoring sont utilisées pour évaluer les candidats à la fusion.

#### Stratégies de fusion

Fast Downward propose plusieurs stratégies de fusion, y compris l'utilisation d'un arbre de fusion précalculé, la stratégie de fusion SSCs, et la stratégie de fusion sans état. Ces stratégies peuvent être personnalisées à l'aide de différents sélecteurs de fusion.

In [19]:
// Exemple d'exécution avec une stratégie de fusion
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(merge_and_shrink(merge_strategy=merge_precomputed(merge_tree=linear(variable_order=reverse_level)),shrink_strategy=shrink_bisimulation(greedy=true),label_reduction=exact(before_shrinking=true,before_merging=false)))\"");


Le fichier Domaine (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/domain.pddl) n'a pas ete trouve.


Le fichier Probleme (C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\misc/tests/benchmarks/gripper/prob01.pddl) n'a pas ete trouve.


INFO     planner time limit: None
INFO     planner memory limit: None

INFO     Running translator.



Erreurs: Could not find build 'release' at C:\Users\jsboi\.local\fast-downward\downward-release-24.06.1\builds\release\bin. Please run './build.py release'.



#### Génération de collections de motifs

Fast Downward propose une série d'algorithmes pour la génération de collections de motifs. Ces algorithmes, y compris la Factory for pattern collections, Combo, Disjoint CEGAR, et d'autres, offrent diverses stratégies pour créer des collections de motifs.

#### Usines à motifs

Fast Downward propose plusieurs usines à motifs, y compris la Factory for single patterns, Greedy, Manual Pattern, et Random Pattern. Chacune de ces usines offre des moyens distincts pour générer des motifs de planification.

En somme, Fast Downward offre une variété d'outils pour personnaliser et optimiser le process


### Soumission aux compétitions internationales

Fast Downward est régulièrement soumis aux compétitions internationales de planification avec un ensemble de configurations complexes à tester sur les problèmes des benchmarks, illustrant l’utilisation simultanée de plusieurs fonctionnalités présentées ci-dessus.

Le [fichier suivant](https://github.com/aibasel/downward/blob/main/misc/tests/configs.py) propose des alias de configuration élémentaires, et [ce répertoire](https://github.com/aibasel/downward/tree/main/driver/portfolios) contient un grand nombre de configurations qui ont été présentées aux compétitions (il faut fusionner les chaînes distinctes en ligne de commande).